# Here I am trying to merge/align the behavioural anotation logs with ACC logs

In [ ]:
import pandas as pd
import numpy as np


ACC_PATH = "../src/test_data/AlgorithmData-Acceleration-[20251001-20251031]-part2.csv"
ANN_PATH = "../src/test_data/11-44-01_2a_annotations.csv"
OUT_PATH = "../src/test_data/acc_labeled_testwindow.csv"

ACC_TIME_COL = "Collecting time"
ANN_START_COL = "start_datetime"
ANN_END_COL = "end_datetime"
ANN_BEHAVIOR_COL = "behavior"

def to_utc(dt_series):
    # Always produce tz-aware UTC timestamps
    return pd.to_datetime(dt_series, errors="coerce", utc=True)

# first load the annoations and convert to UTC tz-aware timestamps
ann = pd.read_csv(ANN_PATH)
ann[ANN_START_COL] = to_utc(ann[ANN_START_COL])
ann[ANN_END_COL] = to_utc(ann[ANN_END_COL])
ann = ann.dropna(subset=[ANN_START_COL, ANN_END_COL, ANN_BEHAVIOR_COL]).sort_values(ANN_START_COL).reset_index(drop=True)

print("Loaded annotations:", len(ann))

window_start = ann[ANN_START_COL].min()
window_end = ann[ANN_END_COL].max()
print("Annotation window:", window_start, "to", window_end)

# Precompute annotation windows as int64 ns for fast, tz-safe comparisons
ann_start_ns = ann[ANN_START_COL].astype("int64").to_numpy()
ann_end_ns   = ann[ANN_END_COL].astype("int64").to_numpy()
ann_beh      = ann[ANN_BEHAVIOR_COL].astype(str).to_numpy()

# processing acc in chunks to avoid memory issues, and labeling each chunk based on the annotation windows
chunksize = 250000
first = True
total_rows = 0

for chunk in pd.read_csv(ACC_PATH, chunksize=chunksize):
    if ACC_TIME_COL not in chunk.columns:
        raise ValueError(f"Column '{ACC_TIME_COL}' not found in accelerometer file. Found: {list(chunk.columns)}")

    chunk[ACC_TIME_COL] = to_utc(chunk[ACC_TIME_COL])
    chunk = chunk.dropna(subset=[ACC_TIME_COL])

    # Restrict to annotation time window 
    chunk = chunk[(chunk[ACC_TIME_COL] >= window_start) & (chunk[ACC_TIME_COL] <= window_end)]
    if len(chunk) == 0:
        continue

    # Convert acc times to int64 ns (tz-safe)
    t_ns = chunk[ACC_TIME_COL].astype("int64").to_numpy()

    labels = np.full(len(chunk), "None", dtype=object)

    # Label rows for each annotation bout
    for i in range(len(ann)):
        mask = (t_ns >= ann_start_ns[i]) & (t_ns <= ann_end_ns[i])
        if mask.any():
            labels[mask] = ann_beh[i]

    chunk["behavior_label"] = labels

    chunk.to_csv(OUT_PATH, index=False, mode="w" if first else "a", header=first)
    first = False
    total_rows += len(chunk)

print("Done.")
print("Rows written:", total_rows)
print("Output saved to:", OUT_PATH)


Loaded annotations: 9
Annotation window: 2025-10-11 11:44:01.133333+00:00 to 2025-10-11 11:45:48.666667+00:00
Done.
Rows written: 10813
Output saved to: ../src/test_data/acc_labeled_testwindow.csv


In [ ]:
# just need to see the temp reolution of the annotation file, so we can match it in the ACC file before merging

import pandas as pd

ANN_PATH = "../src/test_data/11-44-01_2a_annotations.csv"
ann = pd.read_csv(ANN_PATH)

# show all columns so we can see exact start/end values
pd.set_option("display.max_columns", None)

# filter Sitting rows 
sitting = ann[ann["behavior"].astype(str).str.lower() == "sitting"]

print(sitting[["behavior", "start_datetime", "end_datetime"]].to_string(index=False))


behavior             start_datetime               end_datetime
 Sitting 2025-10-11 11:44:01.133333 2025-10-11 11:44:06.733333
 Sitting 2025-10-11 11:44:09.400000 2025-10-11 11:44:54.266667
 Sitting 2025-10-11 11:45:29.733333 2025-10-11 11:45:30.200000
 Sitting 2025-10-11 11:45:34.066667 2025-10-11 11:45:37.200000


# just making some improvements to the older version, such that we get a single file containing only labeled segments of the ACC and an additional file containing all ACC data with marked segments of annotated behaviour


In [7]:
import pandas as pd
import numpy as np

ACC_PATH = "../src/test_data/AlgorithmData-Acceleration-[20251001-20251031]-part2.csv"
ANN_PATH = "../src/test_data/11-44-01_2a_annotations.csv"

OUT_SNIPPET_PATH = "../src/test_data/acc_labeled_testwindow.csv"
OUT_FULL_PATH    = "../src/test_data/acc_with_behavior_full.csv"

ACC_TIME_COL = "Collecting time"
ANN_START_COL = "start_datetime"
ANN_END_COL = "end_datetime"
ANN_BEHAVIOR_COL = "behavior"

def to_utc(dt_series):
    # 
    return pd.to_datetime(dt_series, errors="coerce", utc=True)


ann = pd.read_csv(ANN_PATH)
ann[ANN_START_COL] = to_utc(ann[ANN_START_COL])
ann[ANN_END_COL] = to_utc(ann[ANN_END_COL])
ann = ann.dropna(subset=[ANN_START_COL, ANN_END_COL, ANN_BEHAVIOR_COL]).sort_values(ANN_START_COL).reset_index(drop=True)

print("Loaded annotations:", len(ann))
if len(ann) == 0:
    raise ValueError("No valid annotations found after parsing datetimes/behavior.")

window_start = ann[ANN_START_COL].min()
window_end = ann[ANN_END_COL].max()
print("Annotation window:", window_start, "to", window_end)

# Precompute annotation windows as int64 ns for fast, tz-safe comparisons
ann_start_ns = ann[ANN_START_COL].astype("int64").to_numpy()
ann_end_ns   = ann[ANN_END_COL].astype("int64").to_numpy()
ann_beh      = ann[ANN_BEHAVIOR_COL].astype(str).to_numpy()

# # # # #####
chunksize = 250000

first_snip = True
first_full = True
rows_written_snip = 0
rows_written_full = 0

for chunk in pd.read_csv(ACC_PATH, chunksize=chunksize):
    if ACC_TIME_COL not in chunk.columns:
        raise ValueError(f"Column '{ACC_TIME_COL}' not found in accelerometer file. Found: {list(chunk.columns)}")

    chunk[ACC_TIME_COL] = to_utc(chunk[ACC_TIME_COL])
    chunk = chunk.dropna(subset=[ACC_TIME_COL])
    if len(chunk) == 0:
        continue

    # Convert acc times to int64 ns (tz-safe)
    t_ns = chunk[ACC_TIME_COL].astype("int64").to_numpy()

    # Default label = NA 
    labels_full = np.full(len(chunk), np.nan, dtype=object)

    # Label rows for each annotation bout
    for i in range(len(ann)):
        mask = (t_ns >= ann_start_ns[i]) & (t_ns <= ann_end_ns[i])
        if mask.any():
            labels_full[mask] = ann_beh[i]

    # full FILE
    chunk_full = chunk.copy()
    chunk_full["behavior_label"] = labels_full
    chunk_full.to_csv(
        OUT_FULL_PATH,
        index=False,
        mode="w" if first_full else "a",
        header=first_full
    )
    first_full = False
    rows_written_full += len(chunk_full)

    #
    in_window = (chunk[ACC_TIME_COL] >= window_start) & (chunk[ACC_TIME_COL] <= window_end)
    if in_window.any():
        chunk_snip = chunk.loc[in_window].copy()
        # reuse the already computed labels (subset them to the window)
        chunk_snip["behavior_label"] = np.asarray(labels_full, dtype=object)[in_window.to_numpy()]
        chunk_snip.to_csv(
            OUT_SNIPPET_PATH,
            index=False,
            mode="w" if first_snip else "a",
            header=first_snip
        )
        first_snip = False
        rows_written_snip += len(chunk_snip)

print("Done.")
print("Snippet rows written:", rows_written_snip)
print("Snippet output saved to:", OUT_SNIPPET_PATH)
print("Full rows written:", rows_written_full)
print("Full output saved to:", OUT_FULL_PATH)


Loaded annotations: 9
Annotation window: 2025-10-11 11:44:01.133333+00:00 to 2025-10-11 11:45:48.666667+00:00
Done.
Snippet rows written: 10813
Snippet output saved to: ../src/test_data/acc_labeled_testwindow.csv
Full rows written: 1007474
Full output saved to: ../src/test_data/acc_with_behavior_full.csv


# Just in case we need to modulate time format of the collection time


In [ ]:

import pandas as pd
import re
from pathlib import Path

# here is where fernando can adjust the file path and the hours need to shift
IN_PATH = Path(r"../src/test_data/AlgorithmData-Acceleration-[20251001-20251031]-part2.csv")   
OUT_PATH = IN_PATH.with_name(IN_PATH.stem + "_SHIFTED_Z.csv")

TIME_COL = "Collecting time"  
SHIFT_HOURS = 1                # +1 forward, -2 backward, etc.


ALLOW_BAD_ROWS = True


_iso_z_re_ms_or_more = re.compile(r"^(?P<prefix>.*\.(?P<ms>\d{3}))(?P<extra>\d*)Z$")
_iso_has_fraction = re.compile(r"^(.*\.)\d+(Z)$")

def normalise_to_iso_z(x):
    if x is None:
        return None
    x = str(x).strip()
    if x == "" or x.lower() == "nan":
        return None

    # Standardise separator
    x = x.replace(" ", "T")

    # Convert UTC offsets to Z (handles +00:00 and +0000)
    if x.endswith("+00:00"):
        x = x[:-6] + "Z"
    elif x.endswith("+0000"):
        x = x[:-5] + "Z"
    elif x.endswith("Z") is False:
        # If it has some other timezone, we still try parsing directly later,
        # but most of the data is UTC seemingly
        pass

    # Ensure a fractional part exists (milliseconds). If none, add .000
    if "T" in x and ("." not in x) and x.endswith("Z"):
        x = x[:-1] + ".000Z"

    # If fraction exists but is shorter than 3, pad to 3
    if x.endswith("Z") and "." in x:
        # Split just before Z
        base = x[:-1]
        left, frac = base.split(".", 1)
        # keep only digits in frac
        frac_digits = re.sub(r"\D", "", frac)
        if len(frac_digits) >= 3:
            frac3 = frac_digits[:3]
        else:
            frac3 = frac_digits.ljust(3, "0")
        x = f"{left}.{frac3}Z"

    # If more than 3 digits exist before Z (microseconds), trim to 3
    m = _iso_z_re_ms_or_more.match(x)
    if m:
        x = m.group("prefix") + "Z"

    return x

def format_iso_z_from_utc(dt_utc: pd.Series) -> pd.Series:
    # dt_utc is tz-aware UTC
    # %f = microseconds (6 digits). Slice to milliseconds.
    return dt_utc.dt.strftime("%Y-%m-%dT%H:%M:%S.%f").str.slice(0, 23) + "Z"


df = pd.read_csv(IN_PATH)
if TIME_COL not in df.columns:
    raise KeyError(f"Column '{TIME_COL}' not found. Columns: {df.columns.tolist()}")

raw = df[TIME_COL].astype("string")


norm = raw.map(normalise_to_iso_z)

# Parse as UTC. If strings already have Z -> ok. If some have +00:00 we normalised to Z.
dt = pd.to_datetime(norm, errors="coerce", utc=True)

bad_mask = dt.isna() & raw.notna() & (raw.str.strip() != "") & (raw.str.lower() != "nan")
bad_count = int(bad_mask.sum())

if bad_count and not ALLOW_BAD_ROWS:
    cols_to_show = [c for c in ["UUID", "Transmitting time", TIME_COL] if c in df.columns]
    display(df.loc[bad_mask, cols_to_show].head(50))
    raise ValueError(
        f"{bad_count} rows in '{TIME_COL}' could not be parsed. "
        "Set ALLOW_BAD_ROWS=True to keep them blank, or inspect the displayed rows."
    )


dt_shift = dt + pd.to_timedelta(SHIFT_HOURS, unit="h")


shifted_str = format_iso_z_from_utc(dt_shift)

# Keep blanks where parsing failed / original was blank
shifted_str = shifted_str.where(~dt_shift.isna(), other="")

# Write back
df[TIME_COL] = shifted_str


print("Input:", IN_PATH)
print("Output:", OUT_PATH)
print("Shift hours:", SHIFT_HOURS)
print("Total rows:", len(df))
print("Unparseable timestamps (left blank):", bad_count)

print("\nPreview (first 10):")
display(df[[TIME_COL]].head(10))

df.to_csv(OUT_PATH, index=False)
print("\nSaved")


Input: ..\src\test_data\AlgorithmData-Acceleration-[20251001-20251031]-part2.csv
Output: ..\src\test_data\AlgorithmData-Acceleration-[20251001-20251031]-part2_SHIFTED_Z.csv
Shift hours: 1
Total rows: 1008461
Unparseable timestamps (left blank): 0

Preview (first 10):


,Collecting time
0,2025-10-11T12:40:58.055Z
1,2025-10-11T12:40:58.064Z
2,2025-10-11T12:40:58.074Z
3,2025-10-11T12:40:58.084Z
4,2025-10-11T12:40:58.094Z
5,2025-10-11T12:40:58.104Z
6,2025-10-11T12:40:58.114Z
7,2025-10-11T12:40:58.124Z
8,2025-10-11T12:40:58.134Z
9,2025-10-11T12:40:58.144Z



Saved ✅
